In [1]:
import dash
from dash import html 
from dash import dcc
import plotly.express as px
import plotly.graph_objs as go

import pandas as pd
import numpy as np
from dash.dependencies import Input, Output

In [2]:
external_stylesheets = [
    {
        "href": "https://fonts.googleapis.com/css2?"
                "family=Lato:wght@400;700&display=swap",
        "rel": "stylesheet",
    },
]
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.title = "Avocado Analytics"

In [3]:
data = pd.read_csv("C:\\Users\\Ahmed Mohsen\\Desktop\\Project\\data\\avocado.csv" , index_col=0)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18249 entries, 0 to 11
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          18249 non-null  object 
 1   AveragePrice  18249 non-null  float64
 2   Total Volume  18249 non-null  float64
 3   type          18249 non-null  object 
 4   year          18249 non-null  int64  
 5   region        18249 non-null  object 
dtypes: float64(2), int64(1), object(3)
memory usage: 998.0+ KB


In [5]:
data.head(5)

,Date,AveragePrice,Total Volume,type,year,region
0,2015-12-27,1.33,64236.62,conventional,2015,Albany
1,2015-12-20,1.35,54876.98,conventional,2015,Albany
2,2015-12-13,0.93,118220.22,conventional,2015,Albany
3,2015-12-6,1.08,78992.15,conventional,2015,Albany
4,2015-11-29,1.28,51039.60,conventional,2015,Albany


In [6]:
data["Date"] = pd.to_datetime(data["Date"], format="%Y-%m-%d")
data.sort_values("Date", inplace=True)

In [7]:
data['total_price'] = data["Total Volume"] * data["AveragePrice"]

In [8]:
region = data.groupby(['region'])['Total Volume'].sum().sort_values(ascending=False).reset_index()

In [9]:
data.isna().sum()

Date            0
AveragePrice    0
Total Volume    0
type            0
year            0
region          0
total_price     0
dtype: int64

In [10]:
app.layout = html.Div(
    children=[
        html.Div(
            children=[
                html.P(children="🥑", className="header-emoji"),
                html.H1(
                    children="Avocado Analytics", className="header-title"
                ),
                html.P(
                    children="Analyze the behavior of avocado prices"
                    " and the number of avocados sold in the US"
                    " between 2015 and 2018",
                    className="header-description",
                ),
            ],
            className="header",
        ),
        
        html.Div(
            children=[
                html.Div(
                    children=[
                        html.Div(children="Region", className="menu-title"),
                        dcc.Dropdown(
                            id="region-filter",
                            options=[
                                {"label": region, "value": region}
                                for region in np.sort(data.region.unique())
                            ],
                            value="Albany",
                            clearable=False,
                            className="dropdown",
                        ),
                    ]
                ),
                
                html.Div(
                    children=[
                        html.Div(children="Type", className="menu-title"),
                        dcc.Dropdown(
                            id="type-filter",
                            options=[
                                {"label": avocado_type, "value": avocado_type}
                                for avocado_type in data.type.unique()
                            ],
                            value="organic",
                            clearable=False,
                            searchable=False,
                            className="dropdown",
                        ),
                    ],
                ),
                
                html.Div(
                    children=[
                        html.Div(
                            children="Date Range",
                            className="menu-title"
                            ),
                        dcc.DatePickerRange(
                            id="date-range",
                            min_date_allowed=data.Date.min().date(),
                            max_date_allowed=data.Date.max().date(),
                            start_date=data.Date.min().date(),
                            end_date=data.Date.max().date(),
                        ),
                    ]
                ),
            ],
            className="menu",
        ),
        
        html.Div(
            children=[
                html.Div(
                    children=dcc.Graph(
                        id="price-chart", config={"displayModeBar": False},
                    ),
                    className="card",
                ),
                html.Div(
                    children=dcc.Graph(
                        id="volume-chart", config={"displayModeBar": False},
                    ),
                    className="card",
                ),
            ],
            className="wrapper",
        ),
        
    ]
)

In [11]:
@app.callback(
    [Output("price-chart", "figure"), Output("volume-chart", "figure")],
    [
        Input("region-filter", "value"),
        Input("type-filter", "value"),
        Input("date-range", "start_date"),
        Input("date-range", "end_date"),
    ],
)
def update_charts(region, avocado_type, start_date, end_date):
    mask = (
        (data.region == region)
        & (data.type == avocado_type)
        & (data.Date >= start_date)
        & (data.Date <= end_date)
    )
    filtered_data = data.loc[mask, :]
    price_chart_figure = {
        "data": [
            {
                "x": filtered_data["Date"],
                "y": filtered_data["AveragePrice"],
                "type": "lines",
                "hovertemplate": "$%{y:.2f}<extra></extra>",
            },
        ],
        "layout": {
            "title": {
                "text": "Average Price of Avocados",
                "x": 0.05,
               
                "xanchor": "left",
            },
            "xaxis": {"fixedrange": True},
            "yaxis": {"tickprefix": "$", "fixedrange": True},
            "colorway": ["#17B897"],
        },
    }

    volume_chart_figure = {
        "data": [
            {
                "x": filtered_data["Date"],
                "y": filtered_data["total_price"] ,
                "type": "bar",
            },
        ],
        "layout": {
            "title": {
                "text": "Avocados Sold",
                "x": 0.05,
                "xanchor": "left"
            },
            "xaxis": {"fixedrange": True},
            "yaxis": {"fixedrange": False},
            "colorway": ["#E12D39"],
        },
    }
    return price_chart_figure, volume_chart_figure

In [12]:
app.run_server(port = 9090)

Dash is running on http://127.0.0.1:9090/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:9090/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Oct/2021 13:58:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 13:58:51] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_0_0m1635353916.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 13:58:51] "GET /_dash-component-suites/dash/deps/react@16.v2_0_0m1635353916.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 13:58:51] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_0_0m1635353916.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 13:58:51] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_0_0m1635353916.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 13:58:51] "GET /assets/style.css?m=1635494253.2653067 HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 13:58:51] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_0_0m1635353916.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 13:58:51] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_0_0m